# Calculate mean Intersection-Over-Union (mIOU) metric

A ready-to-use script to find mean Intersection-Over-Union metric of class pairs


**Input**:
- Existing Project (i.e. "london_roads")
- At least one pair of classes (i.e. ("car_gt", "car_lb"))

**Output**:
- intersection, union and IoU for each class pair


## Imports

In [49]:
import supervisely_lib as sly
import os
import collections
from prettytable import PrettyTable
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np

## Configuration

Edit the following settings for your own case

In [50]:
# Change this field to the name of your team, where target workspace exists.
team_name = "The AI Company" # Automatically inserted
# Change this field to the of your workspace, where target project exists.
workspace_name = "Wrinkle and maybe wrinkle" # Automatically inserted
# Change this field to the name of your target project.
project_name = "Deeplab Test" # Automatically inserted

# Configure the following dictionary  so that is will match pairs of ground truth and predicted classes
# between which IOU will be caluclated.
classes_mapping = {
    "Wrinkle": "Wrinkle_dl",
    "Maybe_Wrinkle": "Maybe_Wrinkle_dl",    
}

# If you are running this notebook on a Supervisely web instance, the connection
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

## Script setup

Import nessesary packages and initialize Supervisely API to remotely manage your projects

In [51]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [52]:
team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    raise RuntimeError("Project {!r} not found".format(project_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))
print("Project: id={}, name={}".format(project.id, project.name))

Team: id=18450, name=The AI Company
Workspace: id=25746, name=Wrinkle and maybe wrinkle
Project: id=58115, name=Deeplab Test


## Get Project Meta of Source Project

Project Meta contains information about classes and tags# Get source project meta

In [53]:
meta_json = api.project.get_meta(project.id)
meta = sly.ProjectMeta.from_json(meta_json)

# check that all classes exist
project_classes_names = list(classes_mapping.keys()) + list(classes_mapping.values())

for class_name in project_classes_names:
    if class_name not in meta.obj_classes.keys():
        raise RuntimeError("Class {!r} not found in source project {!r}".format(class_name, project.name))

## Iterate over all images, and calculate metric by annotations pairs

In [54]:
def safe_ratio(num, denom):
    return (num / denom) if denom != 0 else -1

def get_intersection(mask_1, mask_2):
    return (mask_1 & mask_2).sum()


def get_union(mask_1, mask_2):
    return (mask_1 | mask_2).sum()


def get_iou(mask_1, mask_2):
    return safe_ratio(get_intersection(mask_1, mask_2), get_union(mask_1, mask_2))

def _render_labels_for_class_name(labels, class_name, canvas):
    for label in labels:
        if label.obj_class.name == class_name:
            label.geometry.draw(canvas, True)

ious = {}
def add_pair(ann_gt, ann_pred):
    img_size = ann_gt.img_size
    for cls_gt, cls_pred in classes_mapping.items():
        mask_gt, mask_pred = np.full(img_size, False), np.full(img_size, False)
        _render_labels_for_class_name(ann_gt.labels, cls_gt, mask_gt)
        _render_labels_for_class_name(ann_pred.labels, cls_pred, mask_pred)
        iou = get_iou(mask_gt, mask_pred)
        if (cls_gt,cls_pred) not in ious:
            ious[(cls_gt,cls_pred)] = []
        if iou != -1:
            ious[(cls_gt,cls_pred)].append(iou)
        
for batch in sly.batched(images):
    image_ids = [image_info.id for image_info in batch]
    ann_infos = api.annotation.download_batch(dataset.id, image_ids)

    for ann_info in ann_infos:
        ann = sly.Annotation.from_json(ann_info.annotation, meta)
        add_pair(ann, ann)

print(ious)
        

{('Wrinkle', 'Wrinkle_dl'): [0.39731374477501846, 0.5972188025221329, 0.5982396657419775, 0.5844624787384323, 0.49602172506122866, 0.4193001304581383, 0.31558590392512653, 0.0, 0.43670462156731416, 0.41069712821607773, 0.5259824241419333, 0.6098968754809913, 0.3609498545487166, 0.32706014755533325, 0.41788295913044404, 0.5752503114474231, 0.6622159220814852, 0.3445914452690832, 0.5393263439660941, 0.5844607874668554, 0.5335193380796613, 0.4861610278018755, 0.3174449239522216, 0.0, 0.5585373276420417, 0.5920292059628841, 0.4266702532810443, 0.5495646349631614, 0.5517037846218902], ('Maybe_Wrinkle', 'Maybe_Wrinkle_dl'): [0.21674237143306874, 0.2640015724128894, 0.1542340794476527, 0.14231711173706157, 0.09423779602195416, 0.1240259519684772, 0.13917865049763647, 0.0, 0.18956996259052064, 0.13813991574668757, 0.1578037374805153, 0.1425376805760544, 0.2150773503630912, 0.11465414196539608, 0.19469810646659522, 0.18034825870646767, 0.2139286058260588, 0.23809874181443602, 0.2243910290803734

## Print results manually

In [57]:
table = PrettyTable(["classes pair", "metrics values"])

def build_values_text(values):
    return "iou: count: {} mean: {}, std: {}".format(len(values), np.mean(values), np.std(values))
    
for classes, values in ious.items():
    pair_text = "{} <-> {}".format(classes[0], classes[1])
    table.add_row([pair_text, build_values_text(values)])

print(table.get_string())

+------------------------------------+--------------------------------------------------------------------+
|            classes pair            |                           metrics values                           |
+------------------------------------+--------------------------------------------------------------------+
|       Wrinkle <-> Wrinkle_dl       | iou: count: 29 mean: 0.45582040580684774, std: 0.15792706512135607 |
| Maybe_Wrinkle <-> Maybe_Wrinkle_dl | iou: count: 31 mean: 0.16509945408466706, std: 0.07647258997327959 |
+------------------------------------+--------------------------------------------------------------------+


# Done!